In [1]:
import pyrealsense2 as rs
import numpy as np
import cv2
import os
import time
import glob
import yaml

context = rs.context()
devices = context.query_devices()
for dev in devices:
    print(dev.get_info(rs.camera_info.name))

Intel RealSense D405


# 0. 标定
- 标定板
- 相机： Intel RealSense D405 

In [8]:
# 创建管道并配置
pipeline = rs.pipeline()
config = rs.config()

config.enable_stream(rs.stream.color, 1280, 720, rs.format.bgr8, 30)

# 启用红外流并配置Y16格式
# config.enable_stream(rs.stream.infrared, 1, 848, 480, rs.format.y16, 15)
 
# 启动管道
pipe_profile = pipeline.start(config)

In [ ]:
# 获取出厂内参矩阵
profile = pipe_profile.get_stream(rs.stream.color)
intrinsics = profile.as_video_stream_profile().get_intrinsics()
 
print(f"焦距: fx={intrinsics.fx}, fy={intrinsics.fy}")
print(f"主点坐标: ppx={intrinsics.ppx}, ppy={intrinsics.ppy}")
print(f"畸变系数: {intrinsics.coeffs}")

焦距: fx=645.2243041992188, fy=644.3118896484375
主点坐标: ppx=633.8898315429688, ppy=360.0555419921875
畸变系数: [-0.05372624471783638, 0.05638597533106804, 0.000343952386174351, 0.0005959899281151593, -0.01772184669971466]


## 0.1 拍照

In [13]:
# ======================
# 配置参数
# ======================
# SAVE_DIR = "calib_images"
SAVE_DIR = "Apriltag_images"
IMG_PREFIX = "img"
MAX_IMAGES = 30

WIDTH = 1280
HEIGHT = 720
FPS = 30

# ======================
# 创建保存目录
# ======================
os.makedirs(SAVE_DIR, exist_ok=True)

# ======================
# RealSense 初始化
# ======================
pipeline = rs.pipeline()
config = rs.config()

# 只启用 RGB
config.enable_stream(
    rs.stream.color,
    WIDTH,
    HEIGHT,
    rs.format.bgr8,
    FPS
)

print("[INFO] Starting RealSense pipeline...")
profile = pipeline.start(config)

# 等待自动曝光稳定
for _ in range(30):
    pipeline.wait_for_frames()

print("[INFO] Press 'Enter' to save image, 'Esc' to quit")

img_count = 0

try:
    while True:
        frames = pipeline.wait_for_frames()
        color_frame = frames.get_color_frame()
        if not color_frame:
            continue

        color_image = np.asanyarray(color_frame.get_data())

        # 显示提示
        display = color_image.copy()
        cv2.putText(
            display,
            f"Saved: {img_count}/{MAX_IMAGES}",
            (20, 40),
            cv2.FONT_HERSHEY_SIMPLEX,
            1.0,
            (0, 255, 0),
            2
        )
        cv2.putText(
            display,
            "Press 'Enter' to save | 'Esc' to quit",
            (20, HEIGHT - 20),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.8,
            (0, 255, 0),
            2
        )

        cv2.imshow("D405 Calibration Capture", display)
        key = cv2.waitKey(1) & 0xFF

        # 保存图像
        if key == 13:
            filename = f"{IMG_PREFIX}_{img_count:03d}.png"
            path = os.path.join(SAVE_DIR, filename)
            cv2.imwrite(path, color_image)
            print(f"[INFO] Saved {path}")
            img_count += 1
            time.sleep(0.3)  # 防止连拍

        # 退出
        elif key == 27:
            break

        if img_count >= MAX_IMAGES:
            print("[INFO] Reached max images.")
            break

finally:
    pipeline.stop()
    cv2.destroyAllWindows()
    print("[INFO] Pipeline stopped.")


[INFO] Starting RealSense pipeline...
[INFO] Press 'Enter' to save image, 'Esc' to quit
[INFO] Saved Apriltag_images\img_000.png
[INFO] Saved Apriltag_images\img_001.png
[INFO] Pipeline stopped.


## 0.2 计算内参

In [11]:
IMAGE_PATH = "calib_images/*.png"

ROWS = 6   # 棋盘格 内角点 行数（垂直）
COLS = 9   # 棋盘格 内角点 列数（水平）

SQUARE_SIZE =20.0  # mm

OUTPUT_YAML = "d405_rgb_intrinsics.yaml"

# ============================
# 准备世界坐标（Z=0 平面）
# ============================
objp = np.zeros((ROWS * COLS, 3), np.float32)
print(objp.shape)
# print(objp)
objp[:, :2] = np.mgrid[0:COLS, 0:ROWS].T.reshape(-1, 2)
print(objp.shape)
# print(objp)
objp *= SQUARE_SIZE  # 单位：mm
# print(objp)

(54, 3)
(54, 3)


In [12]:
objpoints = []  # 3D 点
imgpoints = []  # 2D 点

# ============================
# 读取图片
# ============================
images = glob.glob(IMAGE_PATH)
assert len(images) > 0, "❌ 没找到标定图片"

print(f"[INFO] Found {len(images)} calibration images")

# ============================
# 棋盘格检测
# ============================
for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    ret, corners = cv2.findChessboardCorners(
        gray,
        (COLS, ROWS),
        cv2.CALIB_CB_ADAPTIVE_THRESH +
        cv2.CALIB_CB_FAST_CHECK +
        cv2.CALIB_CB_NORMALIZE_IMAGE
    )

    if ret:
        objpoints.append(objp)

        # 亚像素精确化
        corners_sub = cv2.cornerSubPix(
            gray,
            corners,
            winSize=(11, 11),
            zeroZone=(-1, -1),
            criteria=(
                cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER,
                30,
                0.001
            )
        )

        imgpoints.append(corners_sub)

        # 可视化（可按 ESC 跳过）
        cv2.drawChessboardCorners(img, (COLS, ROWS), corners_sub, ret)
        cv2.imshow("Corners", img)
        while True:
            key = cv2.waitKey(0) & 0xFF
            if key == 32:      # 空格键
                break
            elif key == 27:    # ESC 键
                cv2.destroyAllWindows()
                exit(0)
    else:
        print(f"[WARN] Chessboard NOT found in {fname}")

cv2.destroyAllWindows()

# ============================
# 相机标定
# ============================
print("[INFO] Calibrating camera...")

ret, camera_matrix, dist_coeffs, rvecs, tvecs = cv2.calibrateCamera(
    objpoints,
    imgpoints,
    gray.shape[::-1],
    None,
    None
)

# ============================
# 计算重投影误差
# ============================
mean_error = 0
for i in range(len(objpoints)):
    imgpoints2, _ = cv2.projectPoints(
        objpoints[i],
        rvecs[i],
        tvecs[i],
        camera_matrix,
        dist_coeffs
    )
    error = cv2.norm(imgpoints[i], imgpoints2, cv2.NORM_L2) / len(imgpoints2)
    mean_error += error

mean_error /= len(objpoints)

print("====== Calibration Result ======")
print("Camera Matrix:\n", camera_matrix)
print("Distortion Coefficients:\n", dist_coeffs.ravel())
print(f"Mean reprojection error: {mean_error:.6f} pixels")

# ============================
# 保存到 YAML
# ============================
data = {
    "camera_matrix": camera_matrix.tolist(),
    "dist_coeffs": dist_coeffs.tolist(),
    "reprojection_error": float(mean_error),
    "image_width": gray.shape[1],
    "image_height": gray.shape[0],
    "square_size_mm": SQUARE_SIZE,
    "board_rows": ROWS,
    "board_cols": COLS
}

with open(OUTPUT_YAML, "w") as f:
    yaml.dump(data, f)

print(f"[INFO] Calibration saved to {OUTPUT_YAML}")


[INFO] Found 28 calibration images
[INFO] Calibrating camera...
====== Calibration Result ======
Camera Matrix:
 [[591.36695928   0.         643.85593567]
 [  0.         590.12810273 370.38761824]
 [  0.           0.           1.        ]]
Distortion Coefficients:
 [-0.05380712  0.04227481  0.00049924  0.00054192 -0.01783079]
Mean reprojection error: 0.025221 pixels
[INFO] Calibration saved to d405_rgb_intrinsics.yaml


# 1. AprilTag

In [ ]:
import cv2
import numpy as np
from pyapriltags import Detector

# =========================
# 参数
# =========================
TAG_SIZE = 0.005  # 5 mm (meters)

camera_params = (
    591.36695928,  # fx
    590.12810273,  # fy
    643.85593567,  # cx
    370.38761824   # cy
)

# =========================
# 初始化 AprilTag Detector
# =========================
detector = Detector(
    families='tag25h9',
    nthreads=2,
    quad_decimate=1.0,
    quad_sigma=0.0,
    refine_edges=1,
    decode_sharpening=0.25
)

# =========================
# 读取图像
# =========================
img = cv2.imread("Apriltag_images/img_001.png", cv2.IMREAD_GRAYSCALE)
assert img is not None, "Image not found!"

# =========================
# AprilTag 检测 + 位姿估计
# =========================
detections = detector.detect(
    img,
    estimate_tag_pose=True,
    camera_params=camera_params,
    tag_size=TAG_SIZE
)

print(f"Detected {len(detections)} tags")

# =========================
# 可视化（一次画完所有 tag）
# =========================
img_color = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)

for det in detections:
    print("Tag ID:", det.tag_id)
    print("Translation t (m):\n", det.pose_t)
    print("Rotation R:\n", det.pose_R)

    corners = det.corners.astype(int)

    # 画边框
    for i in range(4):
        cv2.line(
            img_color,
            tuple(corners[i]),
            tuple(corners[(i + 1) % 4]),
            (0, 255, 0),
            2
        )

    # 写 ID
    cv2.putText(
        img_color,
        f"ID {det.tag_id}",
        tuple(corners[0]),
        cv2.FONT_HERSHEY_SIMPLEX,
        0.6,
        (0, 0, 255),
        2
    )

# =========================
# 显示（按空格继续，ESC 退出）
# =========================
cv2.imshow("AprilTag Pose (Multi-tags)", img_color)

while True:
    key = cv2.waitKey(0) & 0xFF
    if key == 32:      # 空格
        break
    elif key == 27:    # ESC
        cv2.destroyAllWindows()

cv2.destroyAllWindows()


Detected 4 tags
Tag ID: 0
Translation t (m):
 [[-0.02024026]
 [-0.04564775]
 [ 0.13404914]]
Rotation R:
 [[ 0.99077478  0.03526067 -0.13085115]
 [-0.08804925  0.90148257 -0.42376467]
 [ 0.1030178   0.43137669  0.89627087]]
Tag ID: 1
Translation t (m):
 [[ 0.01357844]
 [-0.01465873]
 [ 0.06675172]]
Rotation R:
 [[ 0.99571749  0.08531795 -0.03560236]
 [-0.08210815  0.99311621  0.08353714]
 [ 0.0424845  -0.08025614  0.99586847]]
Tag ID: 2
Translation t (m):
 [[ 0.03257082]
 [-0.01184878]
 [ 0.04569037]]
Rotation R:
 [[ 0.99560028  0.08781345 -0.03269359]
 [-0.08552939  0.99416678  0.06570495]
 [ 0.03827266 -0.0626196   0.99730336]]
Tag ID: 4
Translation t (m):
 [[-2.52684253e-06]
 [ 4.16510338e-03]
 [ 2.57088523e-02]]
Rotation R:
 [[ 0.99483189  0.09174793 -0.04349508]
 [-0.08957467  0.99474779  0.04952995]
 [ 0.04781091 -0.04537792  0.99782512]]
